In [ ]:
import pandas as pd
from emutools.tex import DummyTexDoc, StandardTexDoc
from emutools.utils import add_image_to_doc
from aust_covid.calibration import get_targets
from aust_covid.plotting import plot_multi_spaghetti
from inputs.constants import RUN_IDS, RUNS_PATH, SUPPLEMENT_PATH
import warnings
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

In [ ]:
app_doc = StandardTexDoc(SUPPLEMENT_PATH, 'multi_spaghetti', 'Sequential run plots', 'austcovid')
spaghettis = {k: pd.read_hdf(RUNS_PATH / v / 'output/results.hdf', 'spaghetti') for k, v in RUN_IDS.items()}
targets = get_targets(DummyTexDoc())

In [ ]:
output = 'notifications_ma'
cases_fig = plot_multi_spaghetti(spaghettis, output, targets)
print(output)
cases_fig

In [ ]:
caption = 'Sequential model runs compared against targets as individually coloured lines, one panel for each approach to analysis.'
add_image_to_doc(cases_fig.update_layout(showlegend=False), 'case_spaghetti', 'svg', 'Case time series comparison by run.', app_doc, 'Calibration', caption=caption)

In [ ]:
output = 'deaths_ma'
deaths_fig = plot_multi_spaghetti(spaghettis, output, targets)
print(output)
deaths_fig

In [ ]:
add_image_to_doc(cases_fig.update_layout(showlegend=False), 'death_spaghetti', 'svg', 'Deaths time series comparison by run.', app_doc, 'Calibration', caption=caption)

In [ ]:
output = 'adult_seropos_prop'
seropos_fig = plot_multi_spaghetti(spaghettis, output, targets)
print(output)
seropos_fig

In [ ]:
add_image_to_doc(cases_fig.update_layout(showlegend=False), 'seropos_spaghetti', 'svg', 'Seropositive proportion comparison by run.', app_doc, 'Calibration', caption=caption)

In [ ]:
app_doc.write_doc()